In [1]:
import shap
import joblib
import pandas as pd
import matplotlib.pyplot as plt



IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# Load model and data
model = joblib.load('../models/xgb_churn_model.pkl')
X_test = pd.read_csv('../data/X_test.csv')
y_test = pd.read_csv('../data/y_test.csv').values.ravel()



In [3]:
# Initialize SHAP explainer
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)



[23:57:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


In [4]:
# 1. Summary Plot - Overall feature importance
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test, show=False)
plt.tight_layout()
plt.savefig('../images/shap_summary.png', dpi=300, bbox_inches='tight')
plt.close()



In [5]:
# 2. Bar Plot - Mean absolute SHAP values
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
plt.tight_layout()
plt.savefig('../images/shap_importance.png', dpi=300, bbox_inches='tight')
plt.close()



In [6]:
# 3. Force Plot for a single prediction (high-risk customer)
# Find a customer with high churn probability
predictions = model.predict_proba(X_test)[:, 1]
high_risk_idx = predictions.argmax()

shap.force_plot(
    explainer.expected_value,
    shap_values[high_risk_idx],
    X_test.iloc[high_risk_idx],
    matplotlib=True,
    show=False
)
plt.savefig('../images/shap_force_plot.png', dpi=300, bbox_inches='tight')
plt.close()

# Save explainer for the app
joblib.dump(explainer, '../models/shap_explainer.pkl')

print("✅ SHAP analysis complete!")

✅ SHAP analysis complete!
